---
# Cross-Validation & Backtest Paths
---
Overview of different cross-validation techniques and their application in backtesting financial models. Each technique is demonstrated as follows:

- **Validator Setup**: The `CrossValidatorController` is initialized with `validator_type='VALIDATOR'`, specifying splits and other parameters. Embargo is applied to manage the leakage due to proximity of train and test sets.
- **Split Process**: The data (`X, y`) is split into training and testing indices for each fold using the `.split()` method. The results for each fold, including training and testing indices, are printed.
- **Backtest Paths**: Visualization or analysis of the paths taken during the backtesting phase, showing how the model would perform in a simulated real-world scenario.
- **Backtest Predictions**: Outputs the predictions from backtesting using estimators fitting at each split, providing a comprehensive view of how the model predicts over different folds.

In [28]:
import numpy as np
import pandas as pd
import yfinance as yf
from RiskLabAI.backtest.validation import CrossValidatorController, PurgedKFold
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

## Data
Description and initial processing of the dataset used for demonstrating cross-validation techniques.

In [29]:
# Sample data (for illustration purposes)
n_samples = 120
times_range = pd.date_range(start='01-01-2020', periods=n_samples, freq='D')

data = {
    'Asset 1' : pd.DataFrame({
        'Feature1': np.random.randn(n_samples),
        'Feature2': np.random.randn(n_samples),
        'Target': np.random.choice([0, 1], size=n_samples),
    }, index=times_range), 

    'Asset 2' : pd.DataFrame({
        'Feature1': np.random.randn(n_samples),
        'Feature2': np.random.randn(n_samples),
        'Target': np.random.choice([0, 1], size=n_samples),
    }, index=times_range), 
}

X = {
    'Asset 1' : data['Asset 1'][['Feature1', 'Feature2']],
    'Asset 2' : data['Asset 2'][['Feature1', 'Feature2']],
}

y = {
    'Asset 1' : data['Asset 1']['Target'],
    'Asset 2' : data['Asset 2']['Target'],
}

times = {
    'Asset 1' : pd.Series(times_range + pd.Timedelta('1d'), index=times_range),
    'Asset 2' : pd.Series(times_range + pd.Timedelta('1d'), index=times_range),
}

estimators = {
    'Asset 1' : LogisticRegression(penalty='l2'),
    'Asset 2' : LogisticRegression(penalty='l2'),
}

## Purged Training Times
Explanation and implementation of purging test observations from the training set to prevent information leakage.

In [30]:
PurgedKFold.filtered_training_indices_with_embargo(times['Asset 1'], times['Asset 1'][-20:])

2020-01-01   2020-01-02
2020-01-02   2020-01-03
2020-01-03   2020-01-04
2020-01-04   2020-01-05
2020-01-05   2020-01-06
                ...    
2020-04-04   2020-04-05
2020-04-05   2020-04-06
2020-04-06   2020-04-07
2020-04-07   2020-04-08
2020-04-08   2020-04-09
Length: 99, dtype: datetime64[ns]

## Purged Training Times with Embargo
Details the addition of an embargo period to the purged training times to further ensure no leakage between train and test sets.

In [31]:
PurgedKFold.filtered_training_indices_with_embargo(times['Asset 1'], times['Asset 1'][-20:], embargo_fraction=0.05)

2020-01-01   2020-01-02
2020-01-02   2020-01-03
2020-01-03   2020-01-04
2020-01-04   2020-01-05
2020-01-05   2020-01-06
                ...    
2020-04-04   2020-04-05
2020-04-05   2020-04-06
2020-04-06   2020-04-07
2020-04-07   2020-04-08
2020-04-08   2020-04-09
Length: 99, dtype: datetime64[ns]

## K-Fold
Introduction to the K-Fold cross-validation method, explaining how the dataset is split into K consecutive folds for model validation.

In [32]:
kfold = CrossValidatorController(
    validator_type='kfold',
    n_splits=3,
).cross_validator
for idx, (key, (train_idx, test_idx)) in enumerate(kfold.split(X, y)):
    fold_number = (idx + 1) % kfold.get_n_splits()
    fold_number = kfold.get_n_splits() if fold_number == 0 else fold_number
    print(f"{key}:")
    print(f"Split {fold_number}:")
    print("Train Indices:", train_idx)
    print("Test Indices:", test_idx)
    print("-----")
kfold.backtest_paths(X)
kfold.backtest_predictions(estimators, X, y)

Asset 1:
Split 1:
Train Indices: [ 40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57
  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75
  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93
  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111
 112 113 114 115 116 117 118 119]
Test Indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]
-----
Asset 1:
Split 2:
Train Indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  80  81  82  83  84  85  86  87  88  89  90  91  92  93
  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111
 112 113 114 115 116 117 118 119]
Test Indices: [40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63
 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79]
--

{'Asset 1': {'Path 1': array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0,
         1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1,
         0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
         0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
         0, 1, 0, 0, 1, 1, 0, 1, 0, 0])},
 'Asset 2': {'Path 1': array([0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0,
         0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0,
         0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
         1, 0, 1, 1, 1, 1, 1, 0, 1, 1])}}

## Walk-Forward
Discusses the Walk-Forward validation approach, ensuring that the model is trained and tested on chronological data segments.

In [33]:
walkforward = CrossValidatorController(
    validator_type='walkforward',
    n_splits=3,
).cross_validator
for idx, (key, (train_idx, test_idx)) in enumerate(walkforward.split(X, y)):
    fold_number = (idx + 1) % walkforward.get_n_splits()
    fold_number = walkforward.get_n_splits() if fold_number == 0 else fold_number
    print(f"{key}:")
    print(f"Split {fold_number}:")
    print("Train Indices:", train_idx)
    print("Test Indices:", test_idx)
    print("-----")
walkforward.backtest_paths(X)
walkforward.backtest_predictions(estimators, X, y)

Asset 1:
Split 1:
Train Indices: []
Test Indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]
-----
Asset 1:
Split 2:
Train Indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]
Test Indices: [40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63
 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79]
-----
Asset 1:
Split 3:
Train Indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79]
Test Indices: [ 80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97
  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115
 116 117 118 119]
-----
Asset 2:
Split 1:
Train Indices: []
Test Indices: [ 0  1  2 

{'Asset 1': {'Path 1': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  1.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,
          0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
          0.,  0.,  1.,  1.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  1.,
          1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,
          1.,  0.,  0.])},
 'Asset 2': {'Path 1': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan,  1.,  1.,  1.,  1.,  1.,  0.,

## Purged K-Fold
Combines K-Fold cross-validation with a purging technique to eliminate overlap in the training and testing periods.

In [34]:
purgedkfold = CrossValidatorController(
    validator_type='purgedkfold',
    n_splits=3,
    times=times,
    embargo=0.01
).cross_validator
for idx, (key, (train_idx, test_idx)) in enumerate(purgedkfold.split(X, y)):
    fold_number = (idx + 1) % purgedkfold.get_n_splits()
    fold_number = purgedkfold.get_n_splits() if fold_number == 0 else fold_number
    print(f"{key}:")
    print(f"Split {fold_number}:")
    print("Train Indices:", train_idx)
    print("Test Indices:", test_idx)
    print("-----")
purgedkfold.backtest_paths(X)
purgedkfold.backtest_predictions(estimators, X, y)


Asset 1:
Split 1:
Train Indices: [42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119]
Test Indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]
-----
Asset 1:
Split 2:
Train Indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119]
Test Indices: [40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63
 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79]
--

{'Asset 1': {'Path 1': array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0,
         1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1,
         0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
         0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 1, 1, 0, 1, 0, 0])},
 'Asset 2': {'Path 1': array([0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
         0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0,
         1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0,
         0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 0, 1, 1, 1, 1, 1, 0, 1, 0])}}

## Combinatorial Purged
Explores a combinatorial approach to cross-validation, which considers multiple test periods to optimize the training phase.

In [35]:
combinatorialpurged = CrossValidatorController(
    validator_type='combinatorialpurged',
    n_splits=6,
    n_test_groups=2,
    times=times,
    embargo=0.01
).cross_validator
for idx, (key, (train_idx, test_idx)) in enumerate(combinatorialpurged.split(X, y)):
    fold_number = (idx + 1) % combinatorialpurged.get_n_splits()
    fold_number = combinatorialpurged.get_n_splits() if fold_number == 0 else fold_number
    print(f"{key}:")
    print(f"Split {fold_number}:")
    print("Train Indices:", train_idx)
    print("Test Indices:", test_idx)
    print("-----")
combinatorialpurged.backtest_paths(X)
combinatorialpurged.backtest_predictions(estimators, X, y)

Asset 1:
Split 1:
Train Indices: [42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119]
Test Indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]
-----
Asset 1:
Split 2:
Train Indices: [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119]
Test Indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 40 41 42 43
 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59]


{'Asset 1': {'Path 1': array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0,
         1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0,
         0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
         0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 1, 1, 0, 1, 0, 0]),
  'Path 2': array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
         0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
         1, 1, 0, 1, 1, 1, 0, 1, 0, 0]),
  'Path 3': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
         1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0